### Creating an alternative way to compute indices for wigners

Typically, employing the selection rules for the wigners, we could build a superset of all possible wigners for a given combination of $\ell_1, \ell_2, s, m_1, m_2, m_3$. Hence, efficient storage becomes important as the computation of wigners is expensive and there are huge redundancy because of :

1. symmetries (e.g., cyclic change)
2. parity (in some cases, the wigners differ by only a sign)

This involves the computation of indices (https://github.com/csdms-contrib/slepian_alpha/blob/master/wignersort.m) which assigns a unique index for a unique wigner. However, it is noted that the computation of the index itself is quite expensive, albeit very cheap when compared to the computation of the wigners themselves. For our use-case in `qdPy`, this could potentially be simplified as the wigners always have the same structure. For the problem of rotation, the wigners are always written as $(\ell, s, \ell')$ in the numerator and $(-m, 0, m)$ in the denominator. This ensures that we could eliminate the need to account for the redundancy due to cyclical combinations of the parameters. 

For our case, the uniqueness of a wigner can be attributed to the uniqueness of just 4 parameters. $(\ell, s, \ell', m)$. Defining $\Delta\ell = \ell' - \ell$, we can write the parameters into two sets of pairs $(\ell, m)$ and $(s, \Delta\ell)$. Because of the selection rules, we already know that $\Delta\ell \le s$ and hence, the $(s, \Delta\ell)$ pair could be quantified in the same way as the $(\ell, m)$ pair. It is well known that 
$\ell(\ell+1) + m$ provides a one-dimensional index which is unique for each $(\ell, m)$ pair. Since we have two such pairs, we create a 2D index array with the first index corresponding to $\ell(\ell+1) + s$ and the second index corresponding to $s(s+1) + \Delta\ell$.

The only unique wigners are corresponding to $\ell'>\ell$ and $m\ge 0$. Flipping the sign of the denominator introduces an additional $(-1)^{\ell + \ell' + s}$ factor. Since $s$ is odd and $\Delta\ell$ is even, $s+\ell+\ell' = s+2\ell + \Delta\ell$. Hence only in the case when $s+\Delta\ell$ is odd, a $-1$ factor is introduced.

For odd $s+\Delta\ell$, the $-1$ factor is introduced in the following cases.
\begin{align}
\ell' < \ell & \qquad m \ge 0 &  \qquad f = -1 \\
\ell' > \ell & \qquad  m < 0 & \qquad  f = -1
\end{align}

In [14]:
import numpy as np
def get_wig_idx(ell, s, ellp, m):
    ellc = min(ell, ellp)
    if ellp > ell:
        fac = 1
    else:
        fac = -1
    dell = abs(ellp - ell)
    idx1 = ellc*(ellc+1) + m
    idx2 = s*(s+1) + dell
    return idx1, idx2, fac

The maximum size of the array would be $\frac{\ell_{max}(\ell_{max}+1)}{2}  \frac{s_{max}(s_{max}+1)^2}{2}$

In [15]:
ellmax = 300
smax = 5
sz1 = ellmax*(ellmax+1)/2
sz2 = smax*(smax+1)/2
print(f"Max array shape = ({sz1*sz2}, 2)")
print(f"Total memory for index array = {sz1*sz2*2/1024/1024:.3f} MB")
print(f"Total memory for wigner array = {sz1*sz2*2*8/1024/1024:.3f} MB")

Max array shape = (677250.0, 2)
Total memory for index array = 1.292 MB
Total memory for wigner array = 10.334 MB


In [17]:
result = %timeit -n1 -r1 -o get_wig_idx(200, 3, 201, -100)
time_avg = result.average
time_std = result.stdev
print(f"Time taken for 1.2 billion computations = {time_avg*1.2e9/60:.2f} minutes")

4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Time taken for 1.2 billion computations = 79.92 minutes
